In [2]:
import cv2
import numpy as np


def preprocessing():
    image = cv2.imread("image/image2.png")

    if image is None: return None, None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cv2.imshow("gray1", gray)

    gray = cv2.equalizeHist(gray)

    cv2.imshow("gray2", gray)

    return image, gray


face_cascade = cv2.CascadeClassifier("data/haarcascade_frontalface_alt2.xml")
eys_cascade = cv2.CascadeClassifier("data/haarcascade_eye_tree_eyeglasses.xml")

image, gray = preprocessing()

if image is None: raise Exception("사진 읽기 에러")

faces = face_cascade.detectMultiScale(gray, 1.1, 2, 0, (100, 100))

if faces.any():

    x, y, w, h = faces[0]

    face_image = image[y:y + h, x:x + w]

    eyes = eys_cascade.detectMultiScale(face_image, 1.15, 7, 0, (25, 20))

    if len(eyes) == 2:
        for ex, ey, ew, eh in eyes:
            center = (x + ex + ew // 2, y + ey + eh // 2)
            cv2.circle(image, center, 10, (0, 255, 0), 2)


    else:
        print("눈 미검출")

    cv2.rectangle(image, faces[0], (255, 0, 0), 4)
    cv2.imshow("MyFace", image)

else:
    print("얼굴 미검출")

cv2.waitKey(0)


AttributeError: 'tuple' object has no attribute 'any'

In [18]:
### 사진 기울기 보정

def do_correction_image(image, face_center, eye_centers):
    # 양쪽 눈 좌표
    pt0, pt1 = eye_centers

    if pt0[0] > pt1[0]: pt0, pt1 = pt1, pt0

    # 두 좌표간 차분 계산
    dx, dy = np.subtract(pt1, pt0).astype(float)

    # 역탄젠트로 기울기 계산
    angle = cv2.fastAtan2(dy, dx)

    # 계산된 기울기만큼 이미지 회전하기
    rot = cv2.getRotationMatrix2D(face_center, angle, 1)

    # 회전된 이미지를 원래 이미지 크기로 자르기
    size = image.shape[1::-1]

    # 보정된 이미지 생성
    correction_image = cv2.warpAffine(image, rot, size, cv2.INTER_CUBIC)

    # 눈 위치 보정
    eye_centers = np.expand_dims(eye_centers, axis=0)
    correction_centers = cv2.transform(eye_centers, rot)
    correction_centers = np.squeeze(correction_centers, axis=0)

    return correction_image, correction_centers

# 인식률을 높이기 위한 전처리
def preprocessing():
    # 분석하기 위한 이미지 불러오기
    image = cv2.imread("image/image2.jpg", cv2.IMREAD_COLOR)

    # 이미지가 존재하지 안으면, 에러 반환
    if image is None: return None, None

    # 이미지 크기 사이즈 변경하기
    image = cv2.resize(image, (700, 700))

    # 흑백사진으로 변경
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 변환한 흑백사진으로부터 히스토그램 평활화
    gray = cv2.equalizeHist(gray)

    return image, gray


# 학습된 얼굴 정면검출기 사용하기
face_cascade = cv2.CascadeClassifier("data/haarcascade_frontalface_alt2.xml")

# 학습된 눈 검출기 사용하기
eye_cascade = cv2.CascadeClassifier("data/haarcascade_eye_tree_eyeglasses.xml")

# 인식률을 높이기 위한 전처리 함수 호출
image, gray = preprocessing()  # 전처리

if image is None: raise Exception("영상 파일 읽기 에러")

faces = face_cascade.detectMultiScale(gray, 1.1, 2, 0, (100, 100))

# 얼굴이 검출되었다면,
if faces.any():

    x, y, w, h = faces[0]

    face_image = image[y:y + h, x:x + w]

    eyes = eye_cascade.detectMultiScale(face_image, 1.15, 7, 0, (25, 20))

    # 눈을 찾을 수 있다면,
    if len(eyes) == 2:
        face_center = (int(x + w // 2), int(y + h // 2))

        eye_centers = [[x + ex + ew // 2, y + ey + eh // 2] for ex, ey, ew, eh in eyes]

        correction_image, correction_center = do_correction_image(image, face_center, eye_centers)

        # 보정된 눈 좌표 그리기
        cv2.circle(correction_image, tuple(correction_center[0]), 5, (0, 255, 0), 2)  # 보정 눈 좌표
        cv2.circle(correction_image, tuple(correction_center[1]), 5, (0, 255, 0), 2)  # 보정 눈 좌표

        cv2.circle(correction_image, face_center, 3, (0, 0, 255), 2)  # 얼굴 중심 좌표

        cv2.imshow("MyFace_Edit", correction_image)

    else:
        print("눈 미검출")

else:
    print("얼굴 미검출")

cv2.waitKey(0)

-1